In [4]:
from playwright.async_api import async_playwright, Playwright, Page
from abc import ABC, abstractmethod

async def get_chromium(playwright:Playwright, options:dict=None):
    if options is None:
        options = dict(
            headless=False,
            # headless=True,
            slow_mo=500,
            # args=[
            #     "--start-maximized",
            # ],
        )
    chromium = playwright.chromium
    return await chromium.launch(**options)


async def load_default_page(*, reset_session=False):
    playwright = await start_playwright()
    chrome = await get_chromium(playwright)
    context = None
    try:
        if reset_session:
            context = await chrome.new_context()
        else:
            context = await chrome.new_context(storage_state=".auth/state.json")
    except:
        context = await chrome.new_context()
    page = await context.new_page()
    await page.set_viewport_size({"width": 950, "height": 1000})
    page.set_default_timeout(6000)
    return page


##########################################
#
#          BasePage
#
##########################################
class BasePage(ABC):
    def __init__(self, page:Page):
        self.__page = page
        self.components()
    
    @property
    def page(self):
        return self.__page

    async def goto(self, url):
        await self.__page.goto(self.url())

    async def wait_for_no_overlay(self):
        overlay = self.page.locator('.blockUI.blockOverlay')
        await overlay.wait_for(state="detached", timeout=5000)
    
    @abstractmethod
    def url(self):
        raise ReferenceError("the url is empty")

    @abstractmethod
    def components(self):
        raise ReferenceError("should override components()")